In [ ]:
import cv2
import json
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
from collections import Counter

# Paths
MODEL_PATH = "C://Users//shiba//Downloads//Naruto Project//final_model.keras"
JSON_MAPPING_PATH = "C://Users//shiba//Downloads//Naruto Project//Jutsu_dataset.json"  # Path to Jutsu mapping
CLASS_INDICES_PATH = "C://Users//shiba//Downloads//Naruto Project//class_indices.json"  # Path to class indices
IMG_SIZE = 224
DETECTION_WINDOW_SIZE = 400  # Increased size of the detection window (400x400 pixels)

# Load the trained model
model = load_model(MODEL_PATH)

# Load the JSON mapping for Jutsus
with open(JSON_MAPPING_PATH, 'r') as file:
    jutsu_mapping = json.load(file)

# Load class indices mapping
with open(CLASS_INDICES_PATH, 'r') as file:
    class_indices = json.load(file)

# Reverse the class indices to map index back to class names
index_to_class = {v: k for k, v in class_indices.items()}

# Helper function to preprocess a frame
def preprocess_frame(frame):
    frame_resized = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
    img_array = img_to_array(frame_resized) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Match the sequence to a Jutsu
def match_sequence_to_jutsu(sequence, jutsu_mapping):
    for entry in jutsu_mapping['data']:
        if entry['sequence'] == sequence:
            return entry['jutsu']
    return "Unknown Jutsu"

# Get the most frequent hand sign in the detection window
def get_most_frequent(predictions):
    if predictions:
        return Counter(predictions).most_common(1)[0][0]
    return None

# Check if the center of the hand sign is inside the detection window
def is_hand_in_window(x, y, frame_width, frame_height, window_size=DETECTION_WINDOW_SIZE):
    # Calculate the center of the frame
    center_x = frame_width // 2
    center_y = frame_height // 2
    # Define the window boundaries
    window_left = center_x - window_size // 2
    window_top = center_y - window_size // 2
    window_right = center_x + window_size // 2
    window_bottom = center_y + window_size // 2
    # Check if the hand is within the window
    return window_left <= x <= window_right and window_top <= y <= window_bottom

# Process video and detect Jutsu
def process_video(input_video_path, output_video_path, fps_window=2):
    # Open video
    cap = cv2.VideoCapture(input_video_path)
    
    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frames_per_window = fps * fps_window  # Number of frames in the 1-second window
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    
    # Define output video writer
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
    
    predicted_sequence = []  # Final sequence of hand signs
    window_predictions = []  # Store predictions within the current window
    frame_count = 0  # Track frames within the window
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Preprocess the frame and make predictions
        img_array = preprocess_frame(frame)
        prediction = model.predict(img_array)
        predicted_class_index = np.argmax(prediction, axis=1)[0]
        predicted_class = index_to_class[predicted_class_index]
        
        # Get the center of the hand sign (simulated here with random values; replace with real hand detection)
        # In real application, use hand tracking (e.g., MediaPipe or OpenCV hand tracking)
        # This step should provide the (x, y) coordinates of the hand center.
        hand_x, hand_y = width // 2, height // 2  # Simulating the hand in the center (replace with actual hand detection)
        
        # If the hand is inside the detection window, collect predictions
        if is_hand_in_window(hand_x, hand_y, width, height):
            window_predictions.append(predicted_class)
            frame_count += 1
        
        # If the window is full, determine the most frequent hand sign
        if frame_count >= frames_per_window:
            most_frequent_sign = get_most_frequent(window_predictions)
            if most_frequent_sign and (not predicted_sequence or most_frequent_sign != predicted_sequence[-1]):
                predicted_sequence.append(most_frequent_sign)
            
            # Clear the window for the next segment
            window_predictions = []
            frame_count = 0
        
        # Match the current sequence to a Jutsu
        detected_jutsu = match_sequence_to_jutsu(predicted_sequence, jutsu_mapping)
        
        # Display predictions on the frame
        cv2.putText(frame, f"Hand Sign: {predicted_class}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(frame, f"Sequence: {predicted_sequence}", (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(frame, f"Jutsu: {detected_jutsu}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        
        # Draw the detection window on the frame
        center_x = width // 2
        center_y = height // 2
        cv2.rectangle(frame, (center_x - DETECTION_WINDOW_SIZE // 2, center_y - DETECTION_WINDOW_SIZE // 2),
                      (center_x + DETECTION_WINDOW_SIZE // 2, center_y + DETECTION_WINDOW_SIZE // 2),
                      (0, 255, 0), 2)
        
        # Write the frame to the output video
        out.write(frame)
        
        # Display the frame (optional)
        cv2.imshow("Video", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release resources
    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Example usage
input_video_path = "C://Users//shiba//Downloads//Naruto Project//input.mp4"  # Replace with your input video path
output_video_path = "C://Users//shiba//Downloads//Naruto Project//output_video.avi"  # Replace with your desired output video path

process_video(input_video_path, output_video_path)
print("Video processing complete. Output saved at:", output_video_path)